In [1]:
print("hii")

hii


In [3]:
%pwd

'c:\\Users\\Sanish Bhagat\\project\\derma_ai\\research'

In [4]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
img_size = 224
batch_size = 16
num_classes = 5

train_dir = "../dataset/train"
test_dir = "../dataset/testing"

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.15,
    width_shift_range=0.08,
    height_shift_range=0.08,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    fill_mode="nearest"
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

Found 355 images belonging to 5 classes.


In [7]:
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

Found 85 images belonging to 5 classes.


In [8]:


test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

Found 185 images belonging to 5 classes.


In [9]:
inputs = Input(shape=(img_size, img_size, 3))

base_model = MobileNetV2(
    input_tensor=inputs,
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

C:\Users\Sanish Bhagat\AppData\Local\Temp\ipykernel_19180\889671401.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


In [10]:
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation="softmax")(x)

In [11]:
model = Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [12]:
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        "derma_mobilenet.keras",
        monitor="val_accuracy",
        save_best_only=True
    )
]

In [13]:
history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 29s 837ms/step - accuracy: 0.3690 - loss: 1.7801 - val_accuracy: 0.3882 - val_loss: 1.3824
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 617ms/step - accuracy: 0.5521 - loss: 1.1709 - val_accuracy: 0.5412 - val_loss: 1.3896
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 629ms/step - accuracy: 0.5944 - loss: 1.0730 - val_accuracy: 0.5647 - val_loss: 1.2280
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 663ms/step - accuracy: 0.6563 - loss: 0.9067 - val_accuracy: 0.5529 - val_loss: 1.1198
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 669ms/step - accuracy: 0.7014 - loss: 0.7888 - val_accuracy: 0.5765 - val_loss: 1.1626
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 597ms/step - accuracy: 0.6873 - loss: 0.8679 - val_accuracy: 0.5176 - val_loss: 1.4110
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 631ms/step - accuracy: 0.7099 - loss: 0.7819 - val_accuracy: 0.5176 - val_loss: 1.3978
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 660ms/step - accuracy: 0.6761 - loss: 0.8021 - val_accu

In [ ]:
model.save("../derma_mobilenet.keras")